In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\On-Chip Backward Stimulated Brillouin Scattering in Lithium Niobate Waveguides\\Simulations"
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Postdoc\\Interpolarization Strip LNOI Brillouin Laser"
directory_file_list = os.listdir(directory)
selected_files = []
counter = 0

for i in directory_file_list:
    if i.endswith(".csv"):
        print(f"{counter}: {i}")
        selected_files.append(i)
        counter += 1

In [ ]:
idx_file = 2
df_delimiter = ','
df_header = 4

file_name = selected_files[idx_file]
df_raw = pd.read_csv(directory+"\\"+selected_files[idx_file], delimiter=df_delimiter, header=df_header)
df_raw = df_raw.map(lambda x: complex(str(x).replace('i', 'j')))
columns_raw=df_raw.columns

print(file_name)
df_raw.head()

In [ ]:
idx_df_columns_to_round = [0,1,2]
idx_df_columns_to_delete = []
real_digits = 10
imag_digits = 10
#----------

df = pd.DataFrame(columns=columns_raw)

for i in columns_raw:
    df[i] = df_raw[i]

for i in idx_df_columns_to_round:
    df[columns_raw[i]] = np.round(df_raw[columns_raw[i]].values.real, real_digits) + np.round(df_raw[columns_raw[i]].values.imag, imag_digits)

for i in idx_df_columns_to_delete:
    del df[columns_raw[i]]

columns=df.columns

print(file_name)
df.head()

In [ ]:
print(file_name)
counter = 0
for i in columns:
    print(f"{counter}: {i}")
    counter += 1

In [ ]:
df_filtered = pd.DataFrame(columns=df.columns)
for i in columns:
    df_filtered[i] = df[i]
    
#df_filtered = df_filtered[df_filtered[columns[0]].values.real<550e-9]
#df_filtered = df_filtered[df_filtered[columns[0]].values.real>450e-9]
#df_filtered = df_filtered[abs(df_filtered[columns[31]].values)>0.1]
#df_filtered = df_filtered[abs(df_filtered[columns[31]].values)>0.2]
df_filtered = df_filtered[abs(df_filtered[columns[31]].values)>0.3]

df_filtered.head()

In [ ]:
idx_df_column_X = 1
X = df_filtered[columns[idx_df_column_X]].values

idx_df_column_Y = 2
Y = df_filtered[columns[idx_df_column_Y]].values

idx_df_column_Z = 31
Z = abs(df_filtered[columns[idx_df_column_Z]].values)


fig, ax = plt.subplots(figsize=(10, 6))

im = ax.scatter(X,Y,c=Z,s=10)

ax.set_xlabel(columns[idx_df_column_X])
ax.set_ylabel(columns[idx_df_column_Y])

cbar = ax.figure.colorbar(im, ax=ax, ticks=np.linspace(np.min(Z), np.max(Z), 6))
cbar.set_label(columns[idx_df_column_Z])
cbar.ax.tick_params(labelsize=10)
ax.legend()

ax.grid(True)

print(file_name)

In [32]:
fig.savefig("anglegain.pdf", dpi=600)

# 1 - Group Plotting

In [ ]:
print(file_name)
counter = 0
for column in columns:
    print(f"{counter}: {column}")
    counter += 1

In [10]:
# idx_df_column_X = 0
# idx_df_column_Y = 1
# idx_df_column_Y_groups = 1

# Y_groups = np.unique(df_filtered[columns[idx_df_column_Y_groups]])

# X_spectrogram = []
# Y_spectrogram = []
# fig = plt.figure(figsize=(7, 5))
# flag=0
# for i in Y_groups:
#     X = (df_filtered.groupby(columns[idx_df_column_Y_groups]).get_group(i)[columns[idx_df_column_X]].values.real)
#     Y = abs(df_filtered.groupby(columns[idx_df_column_Y_groups]).get_group(i)[columns[idx_df_column_Y]].values)
#     X_spectrogram.append(X)
#     Y_spectrogram.append(Y)
    
#     plt.scatter(X, Y, label=str(np.round(abs(Y_groups[flag])*1e9, 1))+" [nm]")
#     plt.xlabel(columns[idx_df_column_X])
#     plt.ylabel(columns[idx_df_column_Y])
#     plt.grid(True)
#     plt.legend()
    
#     flag=flag+1
    
# print(selected_files[idx_file])

#fig.savefig("test.pdf", dpi=600)

# 2 - Mechanical Polarization

In [ ]:
print(selected_files[idx_file])
counter = 0
columns = df_filtered.columns
for column in columns:
    print(f"{counter}: {column}")
    counter += 1

In [ ]:
idx_df_column_X = 5
X = df_filtered[columns[idx_df_column_X]].values.real*180/np.pi

idx_df_column_Y = 6
Y = df_filtered[columns[idx_df_column_Y]].values.real/1e9

idx_df_column_mech_xpol = 28
mech_xpol = df_filtered[columns[idx_df_column_mech_xpol]].values.real

idx_df_column_mech_ypol = 29
mech_ypol = df_filtered[columns[idx_df_column_mech_ypol]].values.real

idx_df_column_mech_zpol = 30
mech_zpol = df_filtered[columns[idx_df_column_mech_zpol]].values.real

rgb_vec = [(r, g, b) for r, g, b in zip(mech_xpol, mech_ypol, mech_zpol)]

# create the scatter plot
fig, ax = plt.subplots(figsize=(7,5))

ax.scatter(X, Y, c=rgb_vec, s=5)

ax.set_xlabel("Wavevector [rad/um]")
ax.set_ylabel("Frequency [GHz]")

ax.set_xlim(-60,60)
ax.set_ylim(6, 10.0)

#ax.set_yticks(np.linspace(6.5, 8.5, 11))
#ax.set_xticks([12.69, 13.27, 12.12])

ax.grid(True)

# show the plot
print(selected_files[idx_file])

In [13]:
δf = 0.05 # f0/Q
lorentzian_points = 2000
interp_min = 6.6
interp_max = 10.1

#----------------------
def normalized_lorentzian(f, f0, δf):
    """
    Parameters
    ----------
    f : numpy.ndarray or float
        The frequency where the function is evaluated, in units of Hz
    f0 : float
        The center frequency of the Lorentzian, in units of Hz
    δf : float
        The linewidth of the Lorentzian, expressed as the full width at half maximum (FWHM), in units of Hz
        
    Returns
    -------
    numpy.ndarray or float
        The normalized transmittance value of the function at the given frequency
    """
    return δf**2/(δf**2 + 4*(f - f0)**2)
#----------------------

Z_red_interp = []
Z_green_interp = []
Z_blue_interp = []
red_vec = np.transpose(rgb_vec)[0]
green_vec = np.transpose(rgb_vec)[1]
blue_vec = np.transpose(rgb_vec)[2]
interp_range = np.linspace(interp_min, interp_max, lorentzian_points)
for i in np.arange(len(Y)):
    Z_red_interp.append(red_vec[i]*normalized_lorentzian(interp_range, Y[i], δf))
    Z_green_interp.append(green_vec[i]*normalized_lorentzian(interp_range, Y[i], δf))
    Z_blue_interp.append(blue_vec[i]*normalized_lorentzian(interp_range, Y[i], δf))

# Create a dictionary to store the maximum values for each value in X
red_max_dict = {}
green_max_dict = {}
blue_max_dict = {}

# Loop through each pair of indices where X[i] = X[j] and update the corresponding maximum value in the dictionary
for i in range(len(X)):
    for j in range(i+1, len(X)):
        if X[i] == X[j]:
            red_y_max = [max(y1, y2) for y1, y2 in zip(Z_red_interp[i], Z_red_interp[j])]
            green_y_max = [max(y1, y2) for y1, y2 in zip(Z_green_interp[i], Z_green_interp[j])]
            blue_y_max = [max(y1, y2) for y1, y2 in zip(Z_blue_interp[i], Z_blue_interp[j])]
            
            if X[i] in red_max_dict:
                red_max_dict[X[i]] = [max(red_max_dict[X[i]][k], red_y_max[k]) for k in range(len(red_y_max))]
            else:
                red_max_dict[X[i]] = red_y_max
                
            if X[i] in green_max_dict:
                green_max_dict[X[i]] = [max(green_max_dict[X[i]][k], green_y_max[k]) for k in range(len(green_y_max))]
            else:
                green_max_dict[X[i]] = green_y_max
                
            if X[i] in blue_max_dict:
                blue_max_dict[X[i]] = [max(blue_max_dict[X[i]][k], blue_y_max[k]) for k in range(len(blue_y_max))]
            else:
                blue_max_dict[X[i]] = blue_y_max

idx_unique = np.unique(X, return_index=True)[1]
X_new = X[idx_unique]
Y_new = interp_range
Z_red_new = np.array([red_max_dict[x] if x in red_max_dict else y for x, y in zip(X, Z_red_interp)])[idx_unique]
Z_green_new = np.array([green_max_dict[x] if x in green_max_dict else y for x, y in zip(X, Z_green_interp)])[idx_unique]
Z_blue_new = np.array([blue_max_dict[x] if x in blue_max_dict else y for x, y in zip(X, Z_blue_interp)])[idx_unique]

In [ ]:
print(selected_files[idx_file])
fig, ax = plt.subplots(1,3,figsize=(14,5))

ax[0].pcolormesh(X_new, Y_new, np.transpose(Z_red_new), shading="auto", cmap="Reds")
ax[0].set_title("X Polarization")
ax[0].set_xlabel(columns[idx_df_column_X])
ax[0].set_ylabel(columns[idx_df_column_Y])
ax[0].grid(color="Black", alpha=0.25)

ax[1].pcolormesh(X_new, Y_new, np.transpose(Z_green_new), shading="auto", cmap="Greens")
ax[0].set_title("Y Polarization")
ax[1].set_xlabel(columns[idx_df_column_X])
ax[1].set_ylabel(columns[idx_df_column_Y])
ax[1].grid(color="Black", alpha=0.25)

ax[2].pcolormesh(X_new, Y_new, np.transpose(Z_blue_new), shading="auto", cmap="Blues")
ax[0].set_title("Z Polarization")
ax[2].set_xlabel(columns[idx_df_column_X])
ax[2].set_ylabel(columns[idx_df_column_Y])
ax[2].grid(color="Black", alpha=0.25)

In [15]:
x_points = 2000
y_points = 2000

#----------
# Define the interpolation function
Z_red_new_interp_func = scipy.interpolate.RegularGridInterpolator((X_new, Y_new), Z_red_new, method='linear')
Z_green_new_interp_func = scipy.interpolate.RegularGridInterpolator((X_new, Y_new), Z_green_new, method='linear')
Z_blue_new_interp_func = scipy.interpolate.RegularGridInterpolator((X_new, Y_new), Z_blue_new, method='linear')

# Define a higher-resolution mesh for plotting
X_new_interp = np.linspace(np.min(X_new), np.max(X_new), x_points)
Y_new_interp = np.linspace(np.min(Y_new), np.max(Y_new), y_points)
X_meshgrid, Y_meshgrid = np.meshgrid(X_new_interp, Y_new_interp)
points = np.vstack([X_meshgrid.ravel(), Y_meshgrid.ravel()]).T

# Interpolate the data at the new points
Z_red_new_interp = Z_red_new_interp_func(points).reshape((len(Y_new_interp), len(X_new_interp)))
Z_green_new_interp = Z_green_new_interp_func(points).reshape((len(Y_new_interp), len(X_new_interp)))
Z_blue_new_interp = Z_blue_new_interp_func(points).reshape((len(Y_new_interp), len(X_new_interp)))

Z_new_interp = []
for i in np.arange(len(Z_red_new_interp)):
    Z_new_aux = []
    for j in np.arange(len(Z_red_new_interp[i])):
        Z_new_aux.append([Z_red_new_interp[i][j], Z_green_new_interp[i][j], Z_blue_new_interp[i][j]])
    Z_new_interp.append(Z_new_aux)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_title(selected_files[idx_file])
ax.pcolormesh(X_new_interp, Y_new_interp, Z_new_interp, rasterized=1)
ax.set_xlabel(columns[idx_df_column_X])
ax.set_ylabel(columns[idx_df_column_Y])
ax.set_ylim(1.8, 2.2)
#ax.set_yticks(np.linspace(5.8, 8.2, 9))
#ax.set_xlim(-60, 60)
ax.grid(True)

In [17]:
#fig.savefig("full_polarization_betterQm.pdf", dpi=600)

# 3 - Spectrogram

In [ ]:
print(selected_files[idx_file])
counter = 0
columns = df_filtered.columns
for column in columns:
    print(f"{counter}: {column}")
    counter += 1

In [20]:
idx_df_column_X = 5
X = df_filtered[columns[idx_df_column_X]].values.real

idx_df_column_Y = 6
Y = df_filtered[columns[idx_df_column_Y]].values.real

idx_df_column_Z = 34
Z = abs(df_filtered[columns[idx_df_column_Z]].values)

Qm = 400
δf = 0.01/Qm # f0/Q
lorentzian_points = 2000
interp_min = 6.9e9
interp_max = 8.5e9

#----------------------
def normalized_lorentzian(f, f0, δf):
    """
    Parameters
    ----------
    f : numpy.ndarray or float
        The frequency where the function is evaluated, in units of Hz
    f0 : float
        The center frequency of the Lorentzian, in units of Hz
    δf : float
        The linewidth of the Lorentzian, expressed as the full width at half maximum (FWHM), in units of Hz
        
    Returns
    -------
    numpy.ndarray or float
        The normalized transmittance value of the function at the given frequency
    """
    return δf**2/(δf**2 + 4*(f - f0)**2)
#----------------------

Z_interp = []
interp_range = np.linspace(interp_min, interp_max, lorentzian_points)
for i in np.arange(len(Y)):
    Z_interp.append(Z[i]*normalized_lorentzian(interp_range, Y[i], Y[i]/Qm))
        
# Create a dictionary to store the maximum values for each value in X
max_dict = {}

# Loop through each pair of indices where X[i] = X[j] and update the corresponding maximum value in the dictionary
for i in range(len(X)):
    for j in range(i+1, len(X)):
        if X[i] == X[j]:
            y_max = [max(y1, y2) for y1, y2 in zip(Z_interp[i], Z_interp[j])]
            if X[i] in max_dict:
                max_dict[X[i]] = [max(max_dict[X[i]][k], y_max[k]) for k in range(len(y_max))]
            else:
                max_dict[X[i]] = y_max

# Convert the dictionary to a list of lists to create the new 2D vector Y_new
Z_interp_new = np.array([max_dict[x] if x in max_dict else y for x, y in zip(X, Z_interp)])

idx_unique = np.unique(X, return_index=True)[1]
X_new = X[idx_unique]
Y_new = interp_range
Z_new = Z_interp_new[idx_unique]

In [ ]:
x_points = 2000
y_points = 2000
#----------

print(selected_files[idx_file])
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,5))

cmap0 = plt.cm.RdBu_r
#cmap0 = plt.cm.RdGy_r

im0 = ax[0].pcolormesh(X_new, Y_new, np.transpose(Z_new), cmap=cmap0, zorder=0, rasterized=1)
ax[0].set_title(selected_files[idx_file])
ax[0].set_xlabel(columns[idx_df_column_X])
ax[0].set_ylabel(columns[idx_df_column_Y])
ax[0].grid(True)
cbar0 = ax[0].figure.colorbar(im0, ax=ax[0], ticks=np.linspace(np.min(Z_new), np.max(Z_new), 6), orientation='horizontal')
cbar0.set_label(columns[idx_df_column_Z])
cbar0.ax.tick_params(labelsize=10)

#----------
# Define the interpolation function
Z_new_interp_func = scipy.interpolate.RegularGridInterpolator((X_new, Y_new), Z_new, method='linear')

# Define a higher-resolution mesh for plotting
X_new_interp = np.linspace(np.min(X_new), np.max(X_new), x_points)
Y_new_interp = np.linspace(np.min(Y_new), np.max(Y_new), y_points)
X_meshgrid, Y_meshgrid = np.meshgrid(X_new_interp, Y_new_interp)
points = np.vstack([X_meshgrid.ravel(), Y_meshgrid.ravel()]).T

# Interpolate the data at the new points
Z_new_interp = Z_new_interp_func(points).reshape((len(Y_new_interp), len(X_new_interp)))

# Plot the interpolated data using pcolormesh
im1 = ax[1].pcolormesh(X_new_interp, Y_new_interp, Z_new_interp, cmap=cmap0, zorder=0, rasterized=1)
ax[1].set_title(selected_files[idx_file])
ax[1].set_xlabel(columns[idx_df_column_X])
ax[1].set_ylabel(columns[idx_df_column_Y])
ax[1].grid(True)
cbar1 = ax[1].figure.colorbar(im1, ax=ax[1], ticks=np.linspace(np.min(Z_new_interp), np.max(Z_new_interp), 6), orientation='horizontal')
cbar1.set_label(columns[idx_df_column_Z])
cbar1.ax.tick_params(labelsize=10)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,5))

im = ax.pcolormesh(X_new_interp*(180/np.pi), Y_new_interp/1e9, Z_new_interp, cmap=cmap0, zorder=0, rasterized=1, vmin=0)
ax.set_title(selected_files[idx_file])
ax.set_xlabel("Z-material-axis rotation [degree]")
ax.set_ylabel("Frequency [GHz]")
#ax.set_ylim(6, 10)
#ax.set_yticks(np.linspace(7.8, 8.2, 9))
ax.grid(True)
cbar = ax.figure.colorbar(im, ax=ax, ticks=np.linspace(0, np.max(Z_new_interp), 6))
cbar.set_label("GB/Qm [1/W/m]")
cbar.ax.tick_params(labelsize=10)
print(selected_files[idx_file])

In [22]:
fig.savefig("piezo_clamped_gain_dispersion_TETE.pdf", dpi=600)